# Import Commands

In [1]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
import openpyxl
from qcodes.math_utils import FieldVector
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot

from qcodes_contrib_drivers.drivers.Tektronix.Keithley_2700 import Keithley_2700
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.instrument.visa import VisaInstrument
from qcodes_contrib_drivers.drivers.StanfordResearchSystems.SIM928 import SIM928
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.dataset.plotting import plot_dataset
from qcodes.instrument_drivers.oxford.MercuryiPS_VISA import MercuryiPS
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

# Initialize Instruments

In [3]:
#Magnet Controller
#mag120 = OxfordInstruments_IPS120('ips','TCPIP0::169.254.255.2::7020::SOCKET',23)
#mag120 = OxfordInstruments_IPS120('ips','GPIB0::23::INSTR')
mag120 = MercuryiPS('mips', 'TCPIP0::169.254.255.2::7020::SOCKET')
#Lockins(SR860 and SR830)
rxy2=SR860('Pangolin', 'GPIB1::12::INSTR')
rxx1=SR860('Kiwi','GPIB1::9::INSTR' )
rxy1=SR860('Laughlin', 'GPIB1::4::INSTR')
rxx2=SR830('Wally','GPIB1::1::INSTR' )

Connected to: OXFORD INSTRUMENTS MERCURY IPS (serial:190250049, firmware:2.5.09.000) in 0.05s
Connected to: Stanford_Research_Systems SR860 (serial:004478, firmware:V1.51) in 0.22s
Connected to: Stanford_Research_Systems SR860 (serial:003172, firmware:V1.47) in 0.05s
Connected to: Stanford_Research_Systems SR860 (serial:004479, firmware:V1.51) in 0.04s
Connected to: Stanford_Research_Systems SR830 (serial:s/n48240, firmware:ver1.07) in 0.09s


C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch1_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,
C:\Anaconda3\envs\MattMann\lib\site-packages\qcodes\instrument\instrument_base.py:147: QCoDeSDeprecationWarning: Parameter ch2_databuffer on instrument Wally does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  QCoDeSDeprecationWarning,


# Global Parameters for Instruments

In [11]:
csvPath = "C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11A3\\HallTransport2\\BothHallBars_100pA_Rxy&RxxSweepDown.csv"
dataBasePath = 'C:\\Users\\Admin\Desktop\\MattMann\\8-3-22.1MM11A3\\HallTransport2\\BothHallBars_100pA_Rxy&RxxSweepDown.db'
sample = "8-3-22.1MM11A3"
exp_name = "LargeOhmic Hall Transport"
meas_name = 'LargeOhmic Hall Transport'
amp = 0.000000001
#Sweep rate in T/s
#IMPORTANT: Never go above 50mT/min(0.83mT/s)
sweepRate = 0.00083
fieldTarget = 0

# Set Up Database for Data Collection

In [5]:
db_file_path = os.path.join(os.getcwd(), dataBasePath)
initialise_or_create_database_at(db_file_path)

# Set Magnetic Field Sweep Parameters

In [6]:
#Prints old sweep rate, sets new one, prints new sweep rate
field = mag120._get_ramp_rate()
print("Previous SweepRate")
print(field.z)
rate = FieldVector(0.0,0.0,sweepRate)
mag120._set_ramp_rate(rate)
print("New SweepRate")
print(mag120._get_ramp_rate().z)

Previous SweepRate
0.0008299999999999999
New SweepRate
0.0008299999999999999


In [7]:
#Prints old target field, sets new one, prints new target field
target = FieldVector(0.0,0.0,fieldTarget)
print("Previous Target Field")
print(mag120._get_target_field().z)
mag120._set_target_field(target)
print("New Target Field")
print(mag120._get_target_field().z)

Previous Target Field
7.91
New Target Field
0.0


# Set Up Live Plot

In [8]:
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file_path)

<BackgroundJob #0: <function main at 0x000000001DC65AF8>>

# Setup Station and Measurement Parameters

In [9]:
station = qc.Station()
station.add_component(mag120)
station.add_component(rxy1)
station.add_component(rxx1)
station.add_component(rxy2)
station.add_component(rxx2)

cutoff_freq_exp = load_or_create_experiment(
    experiment_name=exp_name,
    sample_name=sample
)
context_meas = Measurement(exp=cutoff_freq_exp, station=station, name=meas_name)
context_meas.write_period = 0.1

context_meas.register_parameter(mag120.field_measured)
context_meas.register_parameter(rxy1.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxx1.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxy2.X, setpoints=(mag120.field_measured,))
context_meas.register_parameter(rxx2.X, setpoints=(mag120.field_measured,))


# Sweep Magnetic Field and Measure Results for 2 Hall Bars

In [12]:
mag120.ramp('simul')
with context_meas.run() as datasaver:
    while(mag120._get_field().z>0.001):
        set_m = mag120.field_measured.get().z
        get_x1 = rxy1.X.get()/amp
        get_x2 = rxx1.X.get()/amp
        get_x3 = rxy2.X.get()/amp
        get_x4 = rxx2.X.get()/amp

        datasaver.add_result((mag120.field_measured, set_m),
                             (rxy1.X, get_x1),
                             (rxx1.X, get_x2),
                             (rxy2.X, get_x3),
                             (rxx2.X, get_x4))        
        print(set_m)
        time.sleep(1)

    dataset = datasaver.dataset

Starting experimental run with id: 2. 
7.8724
7.8723
7.8713
7.8703
7.8695
7.8684
7.8675
7.8664
7.8652
7.8643
7.8632
7.8623
7.8612
7.8603
7.8592
7.858
7.8571
7.8559
7.8551
7.8539
7.8531
7.8518
7.8509
7.85
7.8488
7.8477
7.8469
7.8456
7.8447
7.8438
7.8426
7.8416
7.8407
7.8394
7.8385
7.8376
7.8364
7.8354
7.8345
7.8333
7.8323
7.831
7.8302
7.8289
7.828
7.827
7.8258
7.8248
7.8239
7.8227
7.8218
7.8208
7.8196
7.8186
7.8177
7.8165
7.8156
7.8147
7.8134
7.8124
7.8115
7.8103
7.8094
7.8085
7.8073
7.8063
7.8054
7.8041
7.8028
7.802
7.8011
7.7998
7.7988
7.7979
7.7967
7.7957
7.7948
7.7936
7.7927
7.7918
7.7905
7.7896
7.7887
7.7874
7.7865
7.7856
7.7843
7.7834
7.7825
7.7812
7.7803
7.7794
7.7782
7.7772
7.7763
7.7751
7.7741
7.7732
7.772
7.771
7.7701
7.7688
7.7679
7.7671
7.7658
7.7648
7.7639
7.7626
7.7618
7.7609
7.7596
7.7587
7.7577
7.7565
7.7556
7.7547
7.7534
7.7525
7.7516
7.7503
7.7494
7.7485
7.7472
7.7463
7.7452
7.7443
7.7433
7.7421
7.7412
7.7402
7.739
7.7381
7.7372
7.7359
7.7351
7.7338
7.7329
7.732
7.7307

6.6534
6.6521
6.6512
6.6503
6.649
6.6481
6.6472
6.6459
6.645
6.644
6.6428
6.6419
6.641
6.6398
6.6388
6.6379
6.6366
6.6357
6.6348
6.6335
6.6326
6.6317
6.6305
6.6295
6.6286
6.6273
6.6264
6.6255
6.6242
6.6233
6.6224
6.6212
6.6204
6.6192
6.6183
6.6173
6.6161
6.6152
6.6142
6.613
6.612
6.6111
6.6099
6.6089
6.608
6.6068
6.6058
6.6049
6.6037
6.6028
6.6019
6.6006
6.5996
6.5987
6.5975
6.5965
6.5956
6.5944
6.5934
6.5925
6.5913
6.5903
6.5894
6.5881
6.5872
6.5864
6.5851
6.5842
6.583
6.582
6.5811
6.5799
6.579
6.5781
6.5768
6.5759
6.5749
6.5737
6.5728
6.5716
6.5704
6.5695
6.5685
6.5673
6.5663
6.5654
6.5642
6.5633
6.5624
6.5611
6.5601
6.5593
6.558
6.5571
6.5562
6.5549
6.554
6.5531
6.5518
6.5509
6.5499
6.5487
6.5478
6.5469
6.5457
6.5447
6.5438
6.5425
6.5416
6.5407
6.5394
6.5385
6.5373
6.5364
6.5355
6.5343
6.5333
6.5322
6.5313
6.5304
6.5291
6.5282
6.5273
6.526
6.5251
6.5242
6.523
6.522
6.5211
6.5198
6.519
6.5178
6.5168
6.5159
6.5147
6.5138
6.5128
6.5116
6.5106
6.5097
6.5085
6.5076
6.5064
6.5055
6.5045
6

5.4258
5.4247
5.4238
5.4226
5.4218
5.4206
5.4197
5.4186
5.4174
5.4166
5.4154
5.4146
5.4134
5.4123
5.4114
5.4102
5.4094
5.4082
5.4074
5.4061
5.4051
5.4042
5.403
5.4022
5.401
5.4001
5.399
5.3978
5.3969
5.3958
5.3949
5.3937
5.3928
5.3917
5.3906
5.3896
5.3885
5.3877
5.3865
5.3856
5.3844
5.3833
5.3825
5.3812
5.3804
5.3793
5.3784
5.3772
5.3761
5.3752
5.374
5.3732
5.372
5.3711
5.37
5.3689
5.368
5.3668
5.3659
5.3648
5.3639
5.3628
5.3616
5.3607
5.3595
5.3587
5.3575
5.3566
5.3555
5.3546
5.3534
5.3524
5.3514
5.3502
5.3495
5.3483
5.3474
5.3462
5.3451
5.3442
5.3431
5.3422
5.3411
5.3402
5.339
5.3379
5.3369
5.336
5.3348
5.3338
5.3329
5.3317
5.3307
5.3298
5.3286
5.3276
5.3267
5.3255
5.3246
5.3236
5.3223
5.3214
5.3205
5.3193
5.3183
5.3175
5.3162
5.3152
5.3143
5.3131
5.3121
5.3112
5.31
5.3091
5.3081
5.3069
5.306
5.3051
5.3038
5.3029
5.302
5.3007
5.2998
5.2989
5.2976
5.2967
5.2957
5.2945
5.2936
5.2927
5.2914
5.2905
5.2895
5.2883
5.2873
5.2864
5.2852
5.2843
5.2834
5.2821
5.2812
5.2802
5.279
5.2781
5.2771


4.196
4.1949
4.194
4.1928
4.1917
4.1909
4.1897
4.1889
4.1877
4.1868
4.1856
4.1845
4.1836
4.1824
4.1816
4.1804
4.1795
4.1783
4.1774
4.1765
4.1754
4.1745
4.1734
4.1722
4.1713
4.1701
4.1693
4.1681
4.1672
4.166
4.165
4.1641
4.1629
4.1621
4.1609
4.16
4.1588
4.1577
4.1568
4.1556
4.1548
4.1536
4.1527
4.1515
4.1505
4.1495
4.1484
4.1476
4.1464
4.1455
4.1444
4.1432
4.1423
4.1412
4.1403
4.1391
4.1383
4.1371
4.136
4.1351
4.1339
4.1332
4.132
4.1311
4.13
4.1294
4.1281
4.127
4.1261
4.125
4.1238
4.123
4.1218
4.1209
4.1198
4.1189
4.1177
4.1166
4.1157
4.1146
4.1137
4.1126
4.1117
4.1105
4.1094
4.1085
4.1073
4.1065
4.1053
4.1044
4.1033
4.1022
4.1013
4.1001
4.0993
4.0981
4.0972
4.0961
4.0949
4.094
4.0928
4.092
4.0909
4.09
4.0888
4.0877
4.0868
4.0856
4.0848
4.0836
4.0828
4.0816
4.0804
4.0796
4.0785
4.0776
4.0764
4.0756
4.0744
4.0732
4.0723
4.0712
4.0703
4.0692
4.0683
4.0671
4.066
4.0651
4.064
4.0631
4.0619
4.061
4.0599
4.059
4.0578
4.0567
4.0558
4.0549
4.0538
4.0529
4.0517
4.0508
4.0497
4.0486
4.0476
4.0465

2.9645
2.9637
2.9625
2.9614
2.9604
2.9595
2.9584
2.9572
2.9561
2.9551
2.9542
2.9531
2.952
2.9509
2.9501
2.9489
2.9478
2.9468
2.9459
2.9449
2.9438
2.943
2.9421
2.941
2.9398
2.9387
2.9379
2.9368
2.9358
2.9346
2.9338
2.9327
2.9316
2.9304
2.9297
2.9285
2.9275
2.9263
2.9255
2.9244
2.9233
2.9222
2.9214
2.9203
2.9192
2.9182
2.9172
2.9162
2.915
2.9142
2.9134
2.9121
2.9113
2.9101
2.9092
2.908
2.9072
2.9062
2.9051
2.9046
2.9032
2.902
2.9009
2.9
2.8989
2.898
2.8969
2.8957
2.8949
2.8937
2.8929
2.8918
2.8906
2.8897
2.8886
2.8877
2.8865
2.8855
2.8846
2.8834
2.8825
2.8815
2.8803
2.8793
2.8783
2.8774
2.8762
2.8751
2.8742
2.8732
2.8721
2.8712
2.8701
2.869
2.8682
2.867
2.8661
2.865
2.8638
2.863
2.8618
2.861
2.8598
2.8586
2.8578
2.8567
2.8559
2.8548
2.8536
2.8534
2.8518
2.8506
2.8498
2.8486
2.8477
2.8466
2.8455
2.8447
2.8435
2.8426
2.8415
2.8403
2.8395
2.8383
2.8381
2.8364
2.8353
2.8344
2.8332
2.8325
2.8313
2.8302
2.8293
2.8281
2.8272
2.8262
2.825
2.8242
2.823
2.8221
2.8216
2.8201
2.8189
2.8181
2.8169
2.

1.7405
1.7394
1.7385
1.7374
1.7365
1.7354
1.7344
1.7333
1.7321
1.7313
1.73
1.7292
1.7281
1.7272
1.726
1.7249
1.7239
1.7229
1.722
1.7208
1.7202
1.7188
1.7177
1.7169
1.7161
1.7149
1.714
1.7132
1.712
1.7108
1.7099
1.7088
1.7079
1.7068
1.7059
1.7047
1.7039
1.703
1.7017
1.7007
1.6998
1.6986
1.6978
1.6966
1.6957
1.6946
1.6934
1.6926
1.6913
1.6905
1.6894
1.6882
1.6873
1.6868
1.6851
1.684
1.6831
1.682
1.6812
1.68
1.6789
1.6779
1.6768
1.676
1.6748
1.6739
1.6728
1.6718
1.6707
1.6696
1.6687
1.6675
1.6667
1.6655
1.6646
1.6635
1.6623
1.6614
1.6602
1.6594
1.6582
1.6573
1.6561
1.6551
1.6547
1.6532
1.6521
1.6512
1.65
1.6492
1.648
1.6471
1.6459
1.6449
1.644
1.6428
1.642
1.6407
1.6401
1.6387
1.6375
1.6367
1.6355
1.635
1.6335
1.6323
1.6315
1.6303
1.6294
1.6283
1.6272
1.6263
1.6253
1.6245
1.6233
1.6227
1.6212
1.6202
1.6192
1.6181
1.6173
1.6161
1.6152
1.6141
1.6129
1.612
1.6108
1.61
1.6088
1.6079
1.6068
1.6057
1.6048
1.6036
1.6028
1.6016
1.6007
1.5996
1.5984
1.5976
1.5964
1.5955
1.5944
1.5935
1.5923
1.5912

0.5141
0.5132
0.5122
0.511
0.51
0.5091
0.5079
0.5071
0.5058
0.5049
0.504
0.5027
0.5018
0.5008
0.4996
0.4987
0.4976
0.4967
0.4958
0.4946
0.4937
0.4928
0.4915
0.4906
0.4897
0.4884
0.4872
0.4863
0.4851
0.4842
0.4832
0.482
0.4811
0.4802
0.4789
0.4779
0.477
0.4758
0.4749
0.474
0.4727
0.4718
0.4706
0.4696
0.4686
0.4675
0.4666
0.4653
0.4645
0.4642
0.4625
0.4613
0.4604
0.4595
0.4582
0.4576
0.4565
0.4553
0.4544
0.4532
0.4523
0.4511
0.4505
0.4491
0.4484
0.4474
0.4462
0.4453
0.444
0.4432
0.442
0.4411
0.4402
0.4389
0.4381
0.4372
0.4359
0.4351
0.4341
0.4331
0.4321
0.4311
0.4302
0.4289
0.428
0.4271
0.4268
0.425
0.4239
0.4229
0.422
0.4207
0.4198
0.4189
0.4177
0.4167
0.4167
0.4147
0.4138
0.4125
0.4116
0.4107
0.4094
0.4085
0.4077
0.4064
0.4054
0.4045
0.4033
0.4023
0.4014
0.4002
0.3996
0.3981
0.3972
0.396
0.395
0.3941
0.3929
0.3919
0.391
0.3899
0.389
0.3878
0.3869
0.3859
0.3847
0.3837
0.3828
0.3816
0.381
0.3798
0.3792
0.3779
0.3766
0.3757
0.3748
0.3736
0.3726
0.3718
0.3705
0.3696
0.3684
0.3674
0.3665
0.

# Converts Dataset Results to CSV

In [13]:
df = dataset.to_pandas_dataframe()
df.to_csv(csvPath)